# merra2

based on daily inputs

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
print('BEGIN')

import socket
import re
import os
import sys
import inspect
import pickle
import glob
#import sort
import datetime
import socket
import re
import os
import sys
import netCDF4
import math

print('sys.version=',sys.version)

hostname=socket.gethostname()

print('hostname=',hostname)

if(re.match('raijin',hostname)):
  print('this is rajin')
  rundir='/short/v14/mac599/cafepp/rundir'
  topdir=''
elif(re.match('oa-3.-cdc',hostname)):
  print('this is vm32')
  rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
  topdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6'
  topdir='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/CMIP6'
  topdir_vm='/OSM/CBR/OA_DCFP/data/CAFEPP'
  topdir_vm='/OSM/CBR/OA_DCFP/work/col414/CAFEPP'
  
os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import numpy as np
import numpy.ma as ma
from matplotlib.mlab import griddata
from scipy import signal
from matplotlib import pyplot as plt
from PyAstronomy import pyaC
import xarray as xr
import windspharm as wsh
import pandas as pd
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib as mpl
import pickle
import bz2

from decadal_diag import \
  fractional_year_from_num2date, \
  generate_daily_month_indices, \
  shade_2d_latlon, \
  get_timestamp_number, \
  shade_2d_curvilinear, \
  shade_2d_simple, \
  nino_indices, \
  plot_xy, \
  fractional_year_from_num2date, \
  convert_bytes, \
  file_size, \
  compress_nc, \
  prepare_vertintp, \
  get_exitcode_stdout_stderr, \
  box_indices, \
  plot_box_indices, \
  basic_stats, \
  lagcorr, \
  turning_points, \
  plot_bar, \
  magnitude
  
from other import year_fraction, \
  cmap_hero, leap_year

from n_data_funcs import n_data_funcs
  
from doppyo import skill
from doppyo import utils
from doppyo import diagnostic
from diagnostic_doppyo_collier import Phillips_criterion, \
  potential_temperature

cafe_experiment_titles = {1:'v0_forecast', 4:'v1_forecast', 3:'v2_forecast', 9:'v1', \
  5:'v2', 6:'v3', 7:'aug17p', 2:'v1_assim', 10:'etkf', 11:'nov17n', 12:'jul18b', 13:'v0'}

fluxes_colors={'hfls':'red', 'hfss':'blue', 'rlds':'green', 'rlus':'orange', \
               'rlut':'pink', 'rsdt':'brown', 'rsus':'cyan', 'rsds':'purple', 'rsut':'grey', \
              'net down toa':'tan', 'net down surf':'yellow'}

%matplotlib inline

CRED = '\033[91m'
CEND = '\033[0m'

#print(cafe_experiment_titles[1])

print('END')

In [ ]:
print('BEGIN')

merra2_time_calendar='proleptic_gregorian'
merra2_time_units='days since 1980-01-01 00:00'

dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}

#AnnOut=False
#AnnOut=True

#ZeroClim=False
#ZeroClim=True

ybeg_merra2=2015
yend_merra2=2016

mbeg_merra2=1
mend_merra2=12

dbeg_merra2=1

if(mend_merra2==2 and leap_year(yend_merra2, merra2_time_calendar)):
  dend_merra2=29
else:
  dend_merra2=dpm[merra2_time_calendar][mend_merra2]

idir_merra2='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/MERRA2'

ifil_merra2_3D=[]
for year in range(ybeg_merra2, yend_merra2+1):
  if(year==ybeg_merra2):
    mbeg=mbeg_merra2
    mend=12
  elif(year==yend_merra2):
    mbeg=1
    mend=mend_merra2
  else:
    mbeg=1
    mend=12 
    
  for month in range(mbeg, mend+1):
    #print('year,month=',year,month)
    YYYY='{0:04d}'.format(year)
    MM='{0:02d}'.format(month)
    ifil_merra2_3D+=glob.glob(idir_merra2+'/'+'MERRA2_?00.inst6_3d_ana_Np.'+YYYY+MM+'??.nc')

input_6hourly_files_merra2_3D=sorted(ifil_merra2_3D)

#print('input_6hourly_files_merra2_3D=',input_6hourly_files_merra2_3D)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_time_stamp_beg=[]
merra2_time_stamp_end=[]
for year in range(ybeg_merra2, yend_merra2+1):
  
  if(not leap_year(year, merra2_time_calendar)):
    days_in_month=dpm[merra2_time_calendar][1:]
  else:
    days_in_month=dpm[merra2_time_calendar][1:]
    days_in_month[1]=29
    
  if(year==ybeg_merra2):
    mbeg=mbeg_merra2
    mend=12
  elif(year==yend_merra2):
    mbeg=1
    mend=mend_merra2
  else:
    mbeg=1
    mend=12
    
  for month in range(mbeg,mend+1):
    
    if(year==ybeg_merra2 and month==mbeg_merra2):
      dbeg=dbeg_merra2
      dend=days_in_month[month-1]
    elif(year==yend_merra2 and month==mend_merra2):
      dbeg=1
      dend=dend_merra2
    else:
      dbeg=1
      dend=days_in_month[month-1]
      if(month==2 and leap_year(year, merra2_time_calendar)):
        dend=29

    for day in range(dbeg,dend+1):
      #print('year,month,day=',year,month,day)
      merra2_time_stamp_beg.append(datetime.datetime(year,month,day) + datetime.timedelta(hours=0.0))
      merra2_time_stamp_end.append(datetime.datetime(year,month,day) + datetime.timedelta(hours=24.0))

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_time_beg = netCDF4.date2num(merra2_time_stamp_beg, merra2_time_units, merra2_time_calendar)
merra2_time_end = netCDF4.date2num(merra2_time_stamp_end, merra2_time_units, merra2_time_calendar)
merra2_time_avg = (merra2_time_beg+merra2_time_end)/2.0
merra2_date_time_stamps = netCDF4.num2date(merra2_time_avg, merra2_time_units, merra2_time_calendar)
merra2_num_time_values = netCDF4.date2num(merra2_date_time_stamps, merra2_time_units, merra2_time_calendar)
merra2_year_fractions = fractional_year_from_num2date(merra2_date_time_stamps, merra2_time_calendar)

#print(merra2_date_time_stamps)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_ifhN_3D = netCDF4.MFDataset(input_6hourly_files_merra2_3D, 'r')

FreshStart=True
#FreshStart=False #turn off initialisation/creation of certain big/timely data objects to speed up.

# try:
#   merra2_ifhN_3D.close()
# else:
#   pass

if(FreshStart):
  merra2_6hourly_U = merra2_ifhN_3D.variables['U']

  merra2_6hourly_lat = merra2_ifhN_3D.variables['lat'][:]
  merra2_6hourly_nlat = merra2_6hourly_lat.size
  merra2_6hourly_lon = merra2_ifhN_3D.variables['lon'][:]
  merra2_6hourly_nlon = merra2_6hourly_lon.size
  merra2_6hourly_lev = merra2_ifhN_3D.variables['lev'][:]
  merra2_6hourly_nlev = merra2_6hourly_lev.size

  merra2_6hourly_U_shape=merra2_6hourly_U.shape

  print('merra2_hourly_U_shape=',merra2_6hourly_U_shape)
  
  if(merra2_6hourly_U_shape[0]!=len(merra2_year_fractions)*4):
    raise SystemExit('Issue with number of times not matching '+str(merra2_6hourly_U_shape[0]/4)+' '+str(len(merra2_year_fractions))+':'+__file__+' line number: '+str(inspect.stack()[0][2]))

  merra2_daily_U=np.average(np.reshape(merra2_6hourly_U[:],[int(merra2_6hourly_U_shape[0]/4), 4, merra2_6hourly_nlev, merra2_6hourly_nlat, merra2_6hourly_nlon]), axis=1) #6-hourly data.

  dataxr_merra2_daily_U = xr.DataArray(merra2_daily_U, \
    coords=[('time', merra2_date_time_stamps.tolist()), \
            ('level', merra2_6hourly_lev), \
            ('lat', merra2_6hourly_lat), \
            ('lon', merra2_6hourly_lon)])

  #print('dataxr_merra2_daily_U=',dataxr_merra2_daily_U)

  merra2_ifhN_3D.close()
  
#delete(merra2_daily_U)
# merra2_year_fractions = year_fraction(merra2_date_time_stamps[0].year, \
#                                    merra2_date_time_stamps[-1].year, \
#                                    mbeg = merra2_date_time_stamps[0].month, \
#                                    mend = merra2_date_time_stamps[-1].month)

#print(merra2_year_fractions)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

levs=(merra2_6hourly_lev*100).astype(int).tolist()

#print('levs=',levs)

merra2_years_months_days_hours=[]
for cnt in range(merra2_date_time_stamps.size):
  merra2_years_months_days_hours.append(merra2_date_time_stamps[cnt].year*1000000+merra2_date_time_stamps[cnt].month*10000+merra2_date_time_stamps[cnt].day*100+merra2_date_time_stamps[cnt].hour)

#print('merra2_years_months_days_hours=',merra2_years_months_days_hours)  

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

lev_to_plot=levs.index(40)
year_months=((2015,9), (2015,10), (2015,11), (2015,12), (2016,1), (2016,2))
year_months=((2016,1), (2016,2), (2016,3), (2016,4), (2016,5), (2016,6), (2016,7), (2016,8), (2016,9), (2016,10), (2016,11), (2016,12))
year_months=((2015,1), (2015,2), (2015,3), (2015,4), (2015,5), (2015,6), (2015,7), (2015,8), (2015,9), (2015,10), (2015,11), (2015,12))

beg_of_months,end_of_months=(),()

for year_month in year_months:
  year,month=year_month
  beg_of_months+=((year,month,1,12),)
  if(leap_year(year) and month==2):
    day_end=29
  else:
    day_end=days_in_month[month-1]
  end_of_months+=((year,month,day_end,12),)

# raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
# beg_of_months=((2015,9,1,12), (2015,10,1,12), (2015,11,1,12), (2015,12,1,12), (2016,1,1,12), (2016,2,1,12))
# end_of_months=((2015,9,30,12), (2015,10,31,12), (2015,11,30,12), (2015,12,31,12), (2016,1,31,12), (2016,2,29,12))

xysize=(6,4)

polygons=([merra2_year_fractions[0],merra2_year_fractions[-1],0,0,'black','--',3],)

for beg_of_month in beg_of_months:
  year,month,day,hour=beg_of_month
  YYYYMMDDHH='{0:04d}'.format(year)+'{0:02d}'.format(month)+'{0:02d}'.format(day)+'{0:02d}'.format(hour)
  #this_data=dataxr_merra2_daily_U.isel(lat=slice(170,191)).mean(axis=-1)
  this_data=dataxr_merra2_daily_U.sel(lat=slice(-5,5)).mean(axis=-1) #atm unweighted for lon (o.k. as spacing even)
  polygons = polygons + ([merra2_year_fractions[merra2_years_months_days_hours.index(int(YYYYMMDDHH))], \
                          merra2_year_fractions[merra2_years_months_days_hours.index(int(YYYYMMDDHH))],-10,10,'green','--',3],) #this is for adding date lines to upcoming xy plot.

for end_of_month in end_of_months:
  year,month,day,hour=end_of_month
  YYYYMMDDHH='{0:04d}'.format(year)+'{0:02d}'.format(month)+'{0:02d}'.format(day)+'{0:02d}'.format(hour)
  polygons = polygons + ([merra2_year_fractions[merra2_years_months_days_hours.index(int(YYYYMMDDHH))], \
                          merra2_year_fractions[merra2_years_months_days_hours.index(int(YYYYMMDDHH))],-10,10,'red','--',3],) #this is for adding date lines to upcoming xy plot.


#xticks=year_fraction(2015,2016)
#xticks=merra2_year_fractions
#xticks=np.arange(2015.5,2016.5+.1,.1).tolist()
xticks = year_fraction(merra2_date_time_stamps[0].year, \
                                   merra2_date_time_stamps[-1].year, \
                                   mbeg = merra2_date_time_stamps[0].month, \
                                   mend = merra2_date_time_stamps[-1].month) #this will give monthly fractions, beginning of each month.

#print(year_fraction(2016,2016,mbeg=1,mend=12))
                                    
xysize=(30,4)

#atm unweighted for lat (o.k. as over a small lat range.)
plot_xy(Diag=False, \
        input_data=(this_data.mean(axis=-1).sel(level=40),), \
        input_xvals=(merra2_year_fractions,), \
        grid=True, \
        title='Merra-2 daily U'+str(dataxr_merra2_daily_U['level'][lev_to_plot]), \
        units='m/s', \
        xysize=xysize, \
        xticks=xticks, \
        xticks_fontsize=8, \
        x_lab_rotation=90, \
#         xlim=[xticks[0],xticks[-1]], \
        xlim=[2015,2016], \
        line_colors=['red'], \
        line_labels=['monthly'], \
        polygons=polygons \
       )

step=10;clevs=sorted(list(set(np.arange(-50, 50+step, step).tolist())))
  
cmap = cmap_hero(clevs, \
              Diag=-1, \
              middle2white=True, \
              negative_infinity='gray', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)
  
xysize=(30,8)

shade_2d_simple(this_data.mean(axis=-1).transpose(), \
  title='Merra-2 daily U', \
  units='m/s', \
  reverse_yaxis=True, \
  yscale='log', \
  xyvals=(merra2_year_fractions, dataxr_merra2_daily_U['level']), \
  xysize=xysize, \
  xlab='Time (Year)', \
  ylab='Pressure (hPa)', \
  ytik=[0.1, 1, 10, 100, 500, 1000], \
#   ytik=[1, 10, 44, 100], \
  ylim=[0.1, 1000], \
#   ylim=[10, 100], \
  xtik=xticks, \
  xlim=[2015,2016], \
  grid=True, \
  cmap=cmap, \
  clevs=clevs, \
  polygons=([merra2_year_fractions[0],merra2_year_fractions[-1],40,40, 'red', '--', 5],), \
  n_contour=[0], \
  colorbar_orientation='horizontal', \
               )

print('END')

In [ ]:
# do same as above but with monthly inputs.

In [ ]:
print('BEGIN')

merra2_monthly_time_calendar='proleptic_gregorian'
merra2_monthly_time_units='days since 1980-01-01 00:00'

dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}

#AnnOut=False
#AnnOut=True

#ZeroClim=False
#ZeroClim=True

SaveClim=False #read from a pkl file.
#SaveClim=True #save to a pkl file useful over large year spans.

ybeg_merra2_monthly=2015 #min 1980,1
yend_merra2_monthly=2016 #max 2018,9

mbeg_merra2_monthly=1
mend_merra2_monthly=12

idir_merra2_monthly='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/MERRA2'

ifil_merra2_monthly_3D=[]
for year in range(ybeg_merra2_monthly, yend_merra2_monthly+1):
  if(year==ybeg_merra2_monthly):
    mbeg=mbeg_merra2_monthly
    mend=12
  elif(year==yend_merra2_monthly):
    mbeg=1
    mend=mend_merra2_monthly
  else:
    mbeg=1
    mend=12 
    
  for month in range(mbeg, mend+1):
    #print('year,month=',year,month)
    YYYY='{0:04d}'.format(year)
    MM='{0:02d}'.format(month)
    ifil_merra2_monthly_3D+=glob.glob(idir_merra2_monthly+'/'+'MERRA2_?00.instM_3d_asm_Np.'+YYYY+MM+'.nc')

input_monthly_files_merra2_3D=sorted(ifil_merra2_monthly_3D)

print('input_monthly_files_merra2_3D=',input_monthly_files_merra2_3D)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_monthly_time_stamp_beg=[]
merra2_monthly_time_stamp_end=[]
for year in range(ybeg_merra2_monthly,yend_merra2_monthly+1):
  if(year==ybeg_merra2_monthly):
    mbeg=1
    mend=12
  elif(year==yend_merra2_monthly):
    mbeg=1
    mend=mend_merra2_monthly
  else:
    mbeg=1
    mend=12    
  for month in range(mbeg,mend+1):
    #print('year,month=',year,month)
    merra2_monthly_time_stamp_beg.append(datetime.datetime(year,month,1,1) + datetime.timedelta(hours=0.0))
    if(month==12):
      merra2_monthly_time_stamp_end.append(datetime.datetime(year+1,1,1) + datetime.timedelta(hours=0.0))
    else:
      merra2_monthly_time_stamp_end.append(datetime.datetime(year,month+1,1) + datetime.timedelta(hours=0.0))
      
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_monthly_time_beg = netCDF4.date2num(merra2_monthly_time_stamp_beg, merra2_monthly_time_units, merra2_monthly_time_calendar)
merra2_monthly_time_end = netCDF4.date2num(merra2_monthly_time_stamp_end, merra2_monthly_time_units, merra2_monthly_time_calendar)
merra2_monthly_time_avg = (merra2_monthly_time_beg+merra2_monthly_time_end)/2.0
merra2_monthly_date_time_stamps = netCDF4.num2date(merra2_monthly_time_avg, merra2_monthly_time_units, merra2_monthly_time_calendar)
merra2_monthly_num_time_values = netCDF4.date2num(merra2_monthly_date_time_stamps, merra2_monthly_time_units, merra2_monthly_time_calendar)
merra2_monthly_year_fractions = fractional_year_from_num2date(merra2_monthly_date_time_stamps, merra2_monthly_time_calendar)

#print('merra2_monthly_date_time_stamps=',merra2_monthly_date_time_stamps)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

merra2_monthly_ifhN_3D = netCDF4.MFDataset(input_monthly_files_merra2_3D, 'r')

FreshStart=True
#FreshStart=False #turn off initialisation/creation of certain big/timely data objects to speed up.

if(FreshStart):
  merra2_monthly_U = merra2_monthly_ifhN_3D.variables['U']
  merra2_monthly_V = merra2_monthly_ifhN_3D.variables['V']

  merra2_monthly_lats = merra2_monthly_ifhN_3D.variables['lat'][:]
  merra2_monthly_nlats = merra2_monthly_lats.size
  merra2_monthly_lons = merra2_monthly_ifhN_3D.variables['lon'][:]
  merra2_monthly_nlons = merra2_monthly_lons.size
  merra2_monthly_levs = merra2_monthly_ifhN_3D.variables['lev'][:]
  merra2_monthly_nlevs = merra2_monthly_levs.size

  merra2_monthly_U_shape=merra2_monthly_U.shape

  #print('merra2_monthly_U_shape=',merra2_monthly_U_shape)
  
  if(merra2_monthly_U_shape[0]!=len(merra2_monthly_year_fractions)):
    raise SystemExit('Issue with number of times not matching '+str(merra2_monthly_U_shape[0])+' '+str(len(merra2_year_fractions))+':'+__file__+' line number: '+str(inspect.stack()[0][2]))

  new_merra2_monthly_lons=merra2_monthly_lons
  new_merra2_monthly_lons=np.where(new_merra2_monthly_lons<0, new_merra2_monthly_lons+360, new_merra2_monthly_lons)
  new_merra2_monthly_lons=np.roll(new_merra2_monthly_lons, 287)

  dataxr_merra2_monthly_U = xr.DataArray(np.roll(merra2_monthly_U[:],287, axis=-1), \
    coords=[('time', merra2_monthly_date_time_stamps.tolist()), \
            ('level', merra2_monthly_levs), \
            ('lat', merra2_monthly_lats), \
            ('lon', new_merra2_monthly_lons)])

  dataxr_merra2_monthly_V = xr.DataArray(np.roll(merra2_monthly_V[:],287, axis=-1), \
  coords=[('time', merra2_monthly_date_time_stamps.tolist()), \
          ('level', merra2_monthly_levs), \
          ('lat', merra2_monthly_lats), \
          ('lon', new_merra2_monthly_lons)])
  
  #print('dataxr_merra2_monthly_U=',dataxr_merra2_monthly_U)

  merra2_monthly_ifhN_3D.close()
  
if(SaveClim):
#   dataxr_merra2_monthly_U_clim = dataxr_merra2_monthly_U.sel(level=slice(40,10)).groupby('time.month').mean(dim='time')
#   dataxr_merra2_monthly_V_clim = dataxr_merra2_monthly_V.sel(level=slice(40,10)).groupby('time.month').mean(dim='time')
  
  dataxr_merra2_monthly_U_clim = dataxr_merra2_monthly_U.groupby('time.month').mean(dim='time')
  dataxr_merra2_monthly_V_clim = dataxr_merra2_monthly_V.groupby('time.month').mean(dim='time')
  
  merra2_monthly_clim_file='merra2_clim_'+str(ybeg_merra2_monthly)+'-'+str(yend_merra2_monthly)+'.pkl'
  if(os.path.exists(merra2_monthly_clim_file)): os.remove(merra2_monthly_clim_file)
  pkl_objects = (dataxr_merra2_monthly_U_clim, dataxr_merra2_monthly_V_clim)
  pickling_out = bz2.BZ2File(merra2_monthly_clim_file, "wb")
  pickle.dump( pkl_objects, pickling_out, protocol=4)
  pickling_out.close()
  raise SystemExit('Getting out after writing out long term climatology:'+__file__+' line number: '+str(inspect.stack()[0][2]))
else:
  pickling_in = bz2.BZ2File('merra2_clim_1980-2000.pkl', "rb")
  dataxr_merra2_monthly_U_clim, dataxr_merra2_monthly_V_clim = pickle.load(pickling_in)
  pickling_in.close()
  
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
dataxr_merra2_monthly_U_anom = utils.anomalize(dataxr_merra2_monthly_U, dataxr_merra2_monthly_U_clim.sel(level=slice(40,10)))

dataxr_merra2_monthly_V_anom = utils.anomalize(dataxr_merra2_monthly_V, dataxr_merra2_monthly_V_clim.sel(level=slice(40,10)))

dataxr_PSI_anom = diagnostic.stream_function(dataxr_merra2_monthly_U_anom, dataxr_merra2_monthly_V_anom) #skip last 3 times as missing.

dataxr_WAF = diagnostic.wave_activity_flux(dataxr_PSI_anom, dataxr_merra2_monthly_U.sel(level=slice(40,10)), dataxr_merra2_monthly_V.sel(level=slice(40,10)))

dataxr_WAF_MAG = magnitude(dataxr_WAF.u_waf, dataxr_WAF.v_waf)

dataxr_U_AVG = dataxr_merra2_monthly_U.mean(dim='level')

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

skip=10

xysize=(12,8)

skip_sp=1
skip_np=1

#step=30;clevs=sorted(list(set(np.arange(-30, 120+step, step).tolist())))
#step=5;clevs=sorted(list(set(np.arange(0, 20+step, step).tolist())))
#step=3;clevs=sorted(list(set(np.arange(0, 12+step, step).tolist())))
clevs=None
cmap='rainbow'
# if(level_choice==0):
#   clevs=[-15,0,15,30,60,90]
#   clevs=[15,30,60,90]
# elif(level_choice==1):
#   clevs=[15,30,60,90]
#   clevs=[5,10,15,20]
# elif(level_choice==2):
#   clevs=[5,10,15,20]
# elif(level_choice==3):

clevs=[5,10,15,20,25]
cmap = cmap_hero(clevs, \
              Diag=-1, \
              first_middle2white=False, \
              negative_infinity='white', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)

step=10;clevs=sorted(list(set(np.arange(-50, 50+step, step).tolist())))
cmap = cmap_hero(clevs, \
              Diag=-1, \
              middle2white=True, \
              negative_infinity='gray', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)

#   dataxr_WAF_MAG.sel(level=40).sel(time=datetime.datetime(2016, 2, 15, 12, 30)), \
#   output=output, \
#   title='merra2 WAF '+YYYYMM+' '+str(int(merra2_monthly_files.lev[levs_selection][xxx][level_choice]))+'hPa', \

#abc

for time_select in range(24):
  year=dataxr_merra2_monthly_U['time.year'][time_select].values
  month=dataxr_merra2_monthly_U['time.month'][time_select].values
  print('time_select,year,month=',time_select,year,month)
  #xxx_time_select=13
  #print(dataxr_WAF['time'].isel(time=xxx_time_select))
  #num_time_select=dataxr_WAF['time'].isel(time=xxx_time_select)
  #dataxr_U_AVG
  #   dataxr_WAF_MAG.sel(level=40).isel(time=xxx_time_select).values, \
  #   dataxr_U_AVG.isel(time=xxx_time_select).values, \
  



  shade_2d_latlon( \
    dataxr_U_AVG.isel(time=time_select).values, \
#     dataxr_WAF_MAG.sel(level=40).isel(time=time_select).values, \
    title='MERRA2 year='+str(year)+' month='+str(month), \
    xysize=xysize, \
    xyvals=(dataxr_WAF['lon'].values, dataxr_WAF['lat'].values), \
    units='$m^2/s^2$', \
    clevs=clevs, \
    cmap=cmap, \
    vector_arrays=((dataxr_WAF['lon'].isel(lon=slice(0,merra2_monthly_nlons,skip)), \
      dataxr_WAF['lat'].isel(lat=slice(skip_sp,merra2_monthly_nlats-skip_np,skip)), \
      dataxr_WAF.u_waf.sel(level=40).isel(time=time_select).isel(lon=slice(0,merra2_monthly_nlons,skip)).isel(lat=slice(skip_sp,merra2_monthly_nlats-skip_np,skip)).values, \
      dataxr_WAF.v_waf.sel(level=40).isel(time=time_select).isel(lon=slice(0,merra2_monthly_nlons,skip)).isel(lat=slice(skip_sp,merra2_monthly_nlats-skip_np,skip)).values, \
      'tail', 800, None),))

raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

levs=(merra2_monthly_levs*100).astype(int).tolist()

#print('levs=',levs)

merra2_monthly_years_months=[]
for cnt in range(merra2_monthly_date_time_stamps.size):
  merra2_monthly_years_months.append(merra2_monthly_date_time_stamps[cnt].year*100+merra2_monthly_date_time_stamps[cnt].month)

#print('merra2_years_months_days_hours=',merra2_years_months_days_hours)  

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

lev_to_plot=levs.index(40)

days_in_month=dpm[merra2_monthly_time_calendar][1:]
  
beg_of_months,end_of_months=(),()
for year in range(ybeg_merra2_monthly,yend_merra2_monthly+1):
  if(year==ybeg_merra2_monthly):
    mbeg=1
    mend=12
  elif(year==yend_merra2_monthly):
    mbeg=1
    mend=mend_merra2_monthly
  else:
    mbeg=1
    mend=12    
  for month in range(mbeg,mend+1):
    #print('year,month=',year,month)
    beg_of_months+=((year,month,1),)
    if(leap_year(year) and month==2):
      day_end=29
    else:
      day_end=days_in_month[month-1]
    end_of_months+=((year,month,day_end),)
  
  datetime.datetime(year+1,1,1) + datetime.timedelta(hours=0.0)
  
#print('beg_of_months=',beg_of_months)
#print('end_of_months=',end_of_months)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

xysize=(6,4)

polygons=([merra2_monthly_year_fractions[0],merra2_monthly_year_fractions[-1],0,0,'black','--',3],)

lons_span=[[260,360]]
#lons_span=[[140,260]]
#lons_span=[[260,360],[0,20]]
      
for beg_of_month in beg_of_months:
  year_beg, month_beg, day_beg=beg_of_month
  #YYYYMM='{0:04d}'.format(year)+'{0:02d}'.format(month)
  #this_data=dataxr_merra2_monthly_U.sel(lat=slice(-10,10)).sel(lon=slice(140,260)).mean(axis=-1)
  
  if(len(lons_span)==1):
    this_data=dataxr_merra2_monthly_U.sel(lat=slice(-5,5)).sel(lon=slice(lons_span[0][0],lons_span[0][1])).mean(axis=-1) #atm unweighted for lon (o.k. as spacing even)
  elif(len(lons_span)==2):
    this_dataA=dataxr_merra2_monthly_U.sel(lat=slice(-5,5)).sel(lon=slice(lons_span[0][0],lons_span[0][1])) #atm unweighted for lon (o.k. as spacing even)
    this_dataB=dataxr_merra2_monthly_U.sel(lat=slice(-5,5)).sel(lon=slice(lons_span[1][0],lons_span[1][1])) #atm unweighted for lon (o.k. as spacing even)
    this_data=xr.concat([this_dataA, this_dataB], dim="lon").mean(axis=-1)
    del(this_dataA,this_dataB)
  
  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
  this_time=datetime.datetime(year_beg, month_beg, day_beg) + datetime.timedelta(hours=0.0)
  this_year_fraction = fractional_year_from_num2date(this_time, merra2_monthly_time_calendar)
  polygons = polygons + ([this_year_fraction, this_year_fraction,-10,10,'green','--',1],) #this is for adding date lines to upcoming xy plot.

for end_of_month in end_of_months:
  year_end, month_end, day_end=end_of_month
  #YYYYMM='{0:04d}'.format(year)+'{0:02d}'.format(month)
  #this_data=dataxr_merra2_monthly_U.sel(lat=slice(-5,5)).mean(axis=-1) #atm unweighted for lon (o.k. as spacing even)
  this_time=datetime.datetime(year_end, month_end, day_end) + datetime.timedelta(hours=24.0)
  this_year_fraction = fractional_year_from_num2date(this_time, merra2_monthly_time_calendar)-.001 #adjust a little so visually can see it.
  #print('this_year_fraction=',this_year_fraction)
  polygons = polygons + ([this_year_fraction, this_year_fraction,-10,10,'red','--',1],) #this is for adding date lines to upcoming xy plot.
    
#xticks=year_fraction(2015,2016)
#xticks=merra2_monthly_year_fractions
#xticks=np.arange(2015.5,2016.5+.1,.1).tolist()
xticks = year_fraction(merra2_monthly_date_time_stamps[0].year, \
                                   merra2_monthly_date_time_stamps[-1].year, \
                                   mbeg = merra2_monthly_date_time_stamps[0].month, \
                                   mend = merra2_monthly_date_time_stamps[-1].month) #this will give monthly fractions, beginning of each month.

#print(year_fraction(2016,2016,mbeg=1,mend=12))
                                    
xysize=(30,4)

#atm unweighted for lat (o.k. as over a small lat range.)
plot_xy(Diag=False, \
        input_data=(this_data.mean(axis=-1).sel(level=40),), \
        input_xvals=(merra2_monthly_year_fractions,), \
        grid=True, \
        title='Merra-2 monthly U'+str(dataxr_merra2_monthly_U['level'][lev_to_plot]), \
        units='m/s', \
        xysize=xysize, \
        xticks=xticks, \
        xticks_fontsize=8, \
        x_lab_rotation=90, \
#         xlim=[xticks[0],xticks[-1]], \
        xlim=[ybeg_merra2_monthly,yend_merra2_monthly+1], \
        line_colors=['red'], \
        line_labels=['monthly'], \
        polygons=polygons \
       )

clevs=None
cmap = 'rainbow'
# step=10;clevs=sorted(list(set(np.arange(-50, 50+step, step).tolist())))
  
# cmap = cmap_hero(clevs, \
#               Diag=-1, \
#               middle2white=True, \
#               negative_infinity='gray', \
#               positive_infinity='pink', \
#               first_cmap='Spectral_r', \
#               first_fraction=[0,.4], \
#               last_cmap='Spectral', \
#               last_fraction=[0,.4], \
#               first_flip=False, \
#               last_flip=True)
  
xysize=(30,8)

shade_2d_simple(this_data.mean(axis=-1).transpose(), \
  title='Merra-2 monthly U', \
  units='m/s', \
  reverse_yaxis=True, \
  yscale='log', \
  xyvals=(merra2_monthly_year_fractions, dataxr_merra2_monthly_U['level']), \
  xysize=xysize, \
  xlab='Time (Year)', \
  ylab='Pressure (hPa)', \
  ytik=[0.1, 1, 10, 100, 500, 1000], \
#   ytik=[1, 10, 44, 100], \
  ylim=[0.1, 1000], \
#   ylim=[10, 100], \
  xtik=xticks, \
  xlim=[ybeg_merra2_monthly,yend_merra2_monthly+1], \
  grid=True, \
  cmap=cmap, \
  clevs=clevs, \
  polygons=([merra2_monthly_year_fractions[0],merra2_monthly_year_fractions[-1],40,40, 'red', '--', 5],), \
  n_contour=[0], \
  colorbar_orientation='horizontal', \
  x_lab_rotation=90 \
               )
print('END')

In [ ]:

#abc

clevs=None
cmap='rainbow'

step=10;clevs=sorted(list(set(np.arange(-50, 50+step, step).tolist())))

cmap = cmap_hero(clevs, \
              Diag=-1, \
              middle2white=True, \
              negative_infinity='gray', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)


#print(dataxr_MMMS)
shade_2d_simple(dataxr_merra2_monthly_U_clim.mean(dim='lon').isel(month=1).values, \
  title='merra2 zonal winds (0-360 average)', \
  units='m/s', \
  cmap=cmap, \
  reverse_yaxis=True, \
  yscale='log', \
  xyvals=(dataxr_merra2_monthly_U_clim['lat'].values, dataxr_merra2_monthly_U_clim['level'].values), \
  xysize=xysize, \
  xlab='Latitude', \
  ylab='Pressure (hPa)', \
  ytik=[1,10,100,1000], \
  ylim=[1,1000], \
  grid=True, \
  clevs=clevs, \
  n_contour=[0], \
  polygons=([-90,90,40,40, \
            'red', \
            '--', \
            5],) \
  )

clevs=None
cmap='rainbow'

step=4;clevs=sorted(list(set(np.arange(-24, 24+step, step).tolist())))

cmap = cmap_hero(clevs, \
              Diag=-1, \
              middle2white=True, \
              negative_infinity='gray', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)

shade_2d_simple(dataxr_merra2_monthly_U.mean(dim='lon').isel(time=0).values - dataxr_merra2_monthly_U_clim.mean(dim='lon').isel(month=0), \
  title='merra2 zonal winds (0-360 average)', \
  units='m/s', \
  cmap=cmap, \
  reverse_yaxis=True, \
  yscale='log', \
  xyvals=(dataxr_merra2_monthly_U_clim['lat'].values, dataxr_merra2_monthly_U_clim['level']), \
  xysize=xysize, \
  xlab='Latitude', \
  ylab='Pressure (hPa)', \
  ytik=[1,10,100,1000], \
  ylim=[1,1000], \
  grid=True, \
  clevs=clevs, \
  n_contour=[0], \
  polygons=([-90,90,40,40, \
            'red', \
            '--', \
            5],) \
  )

In [ ]:
# dataxr_merra2_monthly_U_clim = dataxr_merra2_monthly_U.sel(level=slice(40,10)).groupby('time.month').mean(dim='time')
# dataxr_merra2_monthly_U_anom = utils.anomalize(dataxr_merra2_monthly_U, dataxr_merra2_monthly_U_clim)

# dataxr_merra2_monthly_V_clim = dataxr_merra2_monthly_V.sel(level=slice(40,10)).groupby('time.month').mean(dim='time')
# dataxr_merra2_monthly_V_anom = utils.anomalize(dataxr_merra2_monthly_V, dataxr_merra2_monthly_V_clim)

# dataxr_PSI_anom = diagnostic.stream_function(dataxr_merra2_monthly_U_anom, dataxr_merra2_monthly_V_anom) #skip last 3 times as missing.

# dataxr_WAF = diagnostic.wave_activity_flux(dataxr_PSI_anom, dataxr_merra2_monthly_U.sel(level=slice(40,10)), dataxr_merra2_monthly_V.sel(level=slice(40,10)))

#  time     datetime64[ns] 2016-02-15T12:30:00
      
time_wanted = datetime.datetime(2016,2,15) + datetime.timedelta(hours=12.5)

xxx_time_select=13

#print(dataxr_WAF['time'].isel(time=xxx_time_select))

num_time_select=dataxr_WAF['time'].isel(time=xxx_time_select)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

dataxr_WAF_MAG = magnitude(dataxr_WAF.u_waf, dataxr_WAF.v_waf)

dataxr_U_AVG = dataxr_merra2_monthly_U.mean(dim='level')

#dataxr_WAF_MAG
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#print('dataxr_WAF=',dataxr_WAF)

skip=10

xysize=(12,8)

skip_sp=1
skip_np=1

#step=30;clevs=sorted(list(set(np.arange(-30, 120+step, step).tolist())))
#step=5;clevs=sorted(list(set(np.arange(0, 20+step, step).tolist())))
#step=3;clevs=sorted(list(set(np.arange(0, 12+step, step).tolist())))
clevs=None
cmap='rainbow'
# if(level_choice==0):
#   clevs=[-15,0,15,30,60,90]
#   clevs=[15,30,60,90]
# elif(level_choice==1):
#   clevs=[15,30,60,90]
#   clevs=[5,10,15,20]
# elif(level_choice==2):
#   clevs=[5,10,15,20]
# elif(level_choice==3):

clevs=[5,10,15,20]
cmap = cmap_hero(clevs, \
              Diag=-1, \
              first_middle2white=False, \
              negative_infinity='white', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)

#   dataxr_WAF_MAG.sel(level=40).sel(time=datetime.datetime(2016, 2, 15, 12, 30)), \
#   output=output, \
#   title='merra2 WAF '+YYYYMM+' '+str(int(merra2_monthly_files.lev[levs_selection][xxx][level_choice]))+'hPa', \

shade_2d_latlon( \
  dataxr_WAF_MAG.sel(level=40).sel(time=time_wanted).values, \
#   dataxr_WAF_MAG.sel(level=40).sel(time=num_time_select).values, \
#   dataxr_WAF_MAG.sel(level=40).isel(time=xxx_time_select).values, \
#   dataxr_U_AVG.isel(time=xxx_time_select).values, \
  xysize=xysize, \
  xyvals=(dataxr_WAF['lon'].values, dataxr_WAF['lat'].values), \
  units='$m^2/s^2$', \
  clevs=clevs, \
  cmap=cmap, \
  vector_arrays=((dataxr_WAF['lon'].isel(lon=slice(0,merra2_monthly_nlons,skip)), \
    dataxr_WAF['lat'].isel(lat=slice(0,merra2_monthly_nlats,skip)), \
    dataxr_WAF.u_waf.sel(level=40).isel(time=xxx_time_select).isel(lon=slice(0,merra2_monthly_nlons,skip)).isel(lat=slice(0,merra2_monthly_nlats,skip)).values, \
    dataxr_WAF.v_waf.sel(level=40).isel(time=xxx_time_select).isel(lon=slice(0,merra2_monthly_nlons,skip)).isel(lat=slice(0,merra2_monthly_nlats,skip)).values, \
    'tail', 800, None),))


In [ ]:
#A=this_data
#B=this_data
#C=this_data
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#xysize=(30,4)


#atm unweighted for lat (o.k. as over a small lat range.)
plot_xy(Diag=False, \
        input_data=((B-C).mean(axis=-1).sel(level=40),), \
        input_xvals=(merra2_monthly_year_fractions,), \
        grid=True, \
        title='Merra-2 monthly U'+str(dataxr_merra2_monthly_U['level'][lev_to_plot]), \
        units='m/s', \
        xysize=xysize, \
        xticks=xticks, \
        xticks_fontsize=8, \
        x_lab_rotation=90, \
#         xlim=[xticks[0],xticks[-1]], \
        xlim=[ybeg_merra2_monthly,yend_merra2_monthly+1], \
        line_colors=['red'], \
        line_labels=['monthly'], \
        polygons=polygons \
       )

clevs=None
step=10;clevs=sorted(list(set(np.arange(-50, 50+step, step).tolist())))
  
cmap = cmap_hero(clevs, \
              Diag=-1, \
              middle2white=True, \
              negative_infinity='gray', \
              positive_infinity='pink', \
              first_cmap='Spectral_r', \
              first_fraction=[0,.4], \
              last_cmap='Spectral', \
              last_fraction=[0,.4], \
              first_flip=False, \
              last_flip=True)

#cmap='rainbow'
  
xysize=(30,8)

shade_2d_simple((B-C).mean(axis=-1).transpose(), \
  title='Merra-2 monthly U', \
  units='m/s', \
  reverse_yaxis=True, \
  yscale='log', \
  xyvals=(merra2_monthly_year_fractions, dataxr_merra2_monthly_U['level']), \
  xysize=xysize, \
  xlab='Time (Year)', \
  ylab='Pressure (hPa)', \
  ytik=[0.1, 1, 10, 100, 500, 1000], \
#   ytik=[1, 10, 44, 100], \
  ylim=[0.1, 1000], \
#   ylim=[10, 100], \
  xtik=xticks, \
  xlim=[ybeg_merra2_monthly,yend_merra2_monthly+1], \
  grid=True, \
  cmap=cmap, \
  clevs=clevs, \
  polygons=([merra2_monthly_year_fractions[0],merra2_monthly_year_fractions[-1],40,40, 'red', '--', 5],), \
  n_contour=[0], \
  colorbar_orientation='horizontal', \
  x_lab_rotation=90 \
               )

In [ ]:
# def shade_2d_simple(data,**kwargs):
#   '''
#   plot a 2d array.
#   '''
    
#   import matplotlib.pyplot as plt
#   #https://matplotlib.org/users/colormapnorms.html#custom-normalization-two-linear-ranges
#   #http://chris35wills.github.io/matplotlib_diverging_colorbar/
#   import matplotlib.colors as colors
#   import numpy as np
#   import inspect
#   #from matplotlib.ticker import FormatStrFormatter
#   from matplotlib.ticker import ScalarFormatter, FormatStrFormatter, FixedLocator

#   # set the colormap and centre the colorbar
#   class MidpointNormalize(colors.Normalize):
#     def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
#       self.midpoint = midpoint
#       colors.Normalize.__init__(self, vmin, vmax, clip)

#     def __call__(self, value, clip=None):
#       # I'm ignoring masked values and all kinds of edge cases to make a
#       # simple example...
#       x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
#       return np.ma.masked_array(np.interp(value, x, y))

# #import branca.colormap as cm
# #https://stackoverflow.com/questions/47846744/create-an-asymmetric-colormap
# #colormap = cm.LinearColormap(colors=['red','lightblue','blue'], index=[-3,0,12],vmin=-3,vmax=12)
# #colormap

#   xlab=ylab=xtik=ytik=output=n_contour=contour_arrays=hatching=None
#   Diag = add_contours = title_check = units_check = extend_check = reverse_xaxis = reverse_yaxis = \
#     xyvals_check = xlim_check = ylim_check = grid = xtik_check = ytik_check = cmap_check = \
#     clevs_check = False
#   polygons=None
#   xsize,ysize=6.0,4.0
#   yscale='linear'
#   colorbar_orientation='vertical'
#   x_lab_rotation='horizontal'
#   y_lab_rotation='horizontal'
#   for key, value in kwargs.items():
#     if(Diag): print('key,value=',key,value)
#     if(key=='Diag'):
#       Diag=bool(value)
#     elif(key=='xysize'):
#       xsize,ysize=value
#     elif(key=='clevs'):
#       clevs=value
#       clevs_check=True
#     elif(key=='output'):
#       output=value
#     elif(key=='add_contours'):
#       add_contours=bool(value)
#     elif(key=='title'):
#       title=value
#       title_check=True
#     elif(key=='units'):
#       units=value
#       units_check=True
#     elif(key=='extend'):
#       extend=value
#       extend_check=True
#     elif(key=='reverse_xaxis'):
#       reverse_xaxis=bool(value)
#     elif(key=='reverse_yaxis'):
#       reverse_yaxis=bool(value)
#     elif(key=='xyvals'):
#       xvals,yvals=value
#       xyvals_check=True
#     elif(key=='xlim'):
#       xlim=value
#       xlim_check=True
#     elif(key=='ylim'):
#       ylim=value
#       ylim_check=True
#     elif(key=='xlab'):
#       xlab=value
#       xlab_check=True
#     elif(key=='ylab'):
#       ylab=value
#       ylab_check=True
#     elif(key=='xtik'):
#       xtik=value
#       xtik_check=True
#     elif(key=='ytik'):
#       ytik=value
#       ytik_check=True
#     elif(key=='grid'):
#       grid=bool(value)
#     elif(key=='cmap'):
#       cmap=value
#       cmap_check=True
#     elif(key=='yscale'):
#       yscale=value
#     elif(key=='n_contour'):
#       n_contour=value
#     elif(key=='contour_arrays'):
#       contour_arrays=value
#     elif(key=='polygons'):
#       polygons=value
#     elif(key=='hatching'):
#       hatching,hatching_x,hatching_y,hatching_lev_min,hatching_lev_max=value
#     elif(key=='colorbar_orientation'):
#       colorbar_orientation=value
#     elif(key=='x_lab_rotation'):
#       x_lab_rotation=value
#       if(Diag): print('Rotating x labels.')
#     elif(key=='y_lab_rotation'):
#       y_lab_rotation=value
#       if(Diag): print('Rotating y labels.')
#     else:
#       raise SystemExit('Dont know that key.'+__file__+' line number: '+str(inspect.stack()[0][2]))

#   if(not title_check): title='data'
#   if(not units_check): units='units'  
#   if(not extend_check): extend='both'
#   if(not cmap_check): cmap='jet'
    
#   #print('xvals=',xvals)
#   #print('yvals=',yvals)
  
#   if(not xyvals_check):
#     xvals = np.linspace(0, data.shape[1]-1, data.shape[1])
#     yvals = np.linspace(0, data.shape[0]-1, data.shape[0])
    
#   #print('len(xvals.shape)=',len(xvals.shape))
  
#   #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
#   if(len(xvals.shape)==2):
#     if(len(yvals.shape)!=2):
#       raise SystemExit('Problem with yvals.shape.'+__file__+' line number: '+str(inspect.stack()[0][2]))
#     X=xvals
#     Y=yvals
#   else:
#     (X, Y) = np.meshgrid(xvals, yvals)

#   #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

# #   print('X.shape=',X.shape)
# #   print('Y.shape=',Y.shape)
# #   print('data.shape=',data.shape)
  
#   fig,ax=plt.subplots()
#   fig.set_size_inches(xsize, ysize)

#   #fig = plt.gcf()
#   #fig.set_size_inches(xsize, ysize) #default 6.0,4.0

#   #print('clevs=',clevs)
#   if(not clevs_check or type(clevs)==type(None)):
#     cs=plt.contourf(X, Y, data, extend=extend, cmap=cmap) #good
# #     cs=plt.pcolormesh(X, Y, data, cmap=cmap)
#   else:
#     cs=plt.contourf(X, Y, data, clevs, extend=extend, cmap=cmap) #good

#   #print(hatching)
#   if(type(hatching)!=type(None)):
#     #print('hatching')
#     ch=plt.contourf(hatching_x, hatching_y, hatching, levels=[hatching_lev_min, hatching_lev_max], colors='none', \
#       hatches=['.', '/', '\\', None, '\\\\', '*'], extend='lower')
    
#   if(type(n_contour)!=type(None)):
#     nc=plt.contour(X, Y, data, levels=n_contour, colors='black')
#     plt.clabel(nc, inline=False, fontsize=10, fmt='%3.0f') #, manual=manual_locations)

#   if(type(contour_arrays)!=type(None)):
#     colours=['black','pink','green','orange','purple','cyan']
#     #print('xxx',len(contour_arrays))
#     for cnt in range(len(contour_arrays)):
#       anotherX,anotherY,anotherdata=contour_arrays[cnt]

#       #ca=plt.contour(anotherX, anotherY, anotherdata, colors=colours[cnt], levels=n_contour, linewidths=[1,1,1,5,1,1,1,1,1]) #something odd when I specify values for each level...
#       ca=plt.contour(anotherX, anotherY, anotherdata, colors=colours[cnt], levels=n_contour)
#       #print('xxx',plt.rcParams['lines.linewidth']) #this gives linewidth.

#       plt.clabel(ca, inline=False, fontsize=10, fmt='%3.0f') #, manual=manual_locations)

#     #raise SystemExit('Forced exit file:'+__file__+' line number: '+str(inspect.stack()[0][2]))

# #     cs=plt.pcolormesh(X, Y, data, clevs, cmap=cmap)
#     #cs=plt.contourf(X, Y, data, extend=extend, norm=MidpointNormalize(midpoint=0.,vmin=-1,vmax=1), cmap=cmap)
#     #cs=plt.contourf(X, Y, data, extend=extend, norm=MidpointNormalize(midpoint=0.,vmin=-5,vmax=20), cmap=cmap)
#     #cs=plt.contourf(X, Y, data, extend=extend, norm=MidpointNormalize(midpoint=0.,vmin=np.min(data),vmax=np.max(data)), cmap=cmap)
  
#   if(add_contours): plt.contour(X, Y, data,colors='black')
# #plt.colorbar(fill, orientation='horizontal')
#   cb=plt.colorbar(cs, orientation=colorbar_orientation) #,extend=extend)
#   cb.set_label(units)
  
#   ###
  
#   if(type(title)!=type(None)):
#     if(type(units)==type(None)):
#       plt.title(title, fontsize=16)
#     else:
#       plt.title(title+' ('+units+')', fontsize=16)

#   if(xlim_check): plt.xlim(xlim)
      
#   if(ylim_check): plt.ylim(ylim)
          
#   if(reverse_xaxis): plt.gca().invert_xaxis()
      
#   if(reverse_yaxis): plt.gca().invert_yaxis()
    
#   if(type(xlab)!=type(None)): plt.xlabel(xlab)

#   if(type(ylab)!=type(None)): plt.ylabel(ylab)
    
#   #plt.ylim((0,1000))
#   #https://matplotlib.org/gallery/scales/symlog_demo.html#sphx-glr-gallery-scales-symlog-demo-py
#   #plt.yscale('symlog')

#   plt.yscale(yscale)

#   if(yscale!='linear'):
#     #minorLocator = FixedLocator([100,500,900])
#     #ax.yaxis.set_major_locator(minorLocator)
#     ax.yaxis.set_major_formatter(ScalarFormatter())
#     ax.yaxis.set_major_formatter(FormatStrFormatter("%d"))
#     #ax.tick_params(axis='y', which='minor')
#     #ax.yaxis.set_minor_formatter(FormatStrFormatter("%.0f"))
  
#   if(grid): plt.grid(True,linestyle='-')
   
#   if(type(polygons)!=type(None)):
#     for ppp in range(len(polygons)):
#       #print('ppp=',ppp)
#       ax.plot(np.array([polygons[ppp][0],polygons[ppp][1]]), \
#               np.array([polygons[ppp][2],polygons[ppp][3]]), \
#               color=polygons[ppp][4], \
#               linestyle=polygons[ppp][5], \
#               linewidth=polygons[ppp][6], \
#              )
      
#       #raise SystemExit('Forced exit file:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
#   locs, labels = plt.xticks()
  
#   #plt.xticks(range(-90, 90, 30), fontsize=14)
# #  if(type(xtik)!=type(None)): plt.xticks(xtik)  

#   #print('len(ytik)=',len(ytik))
#   if(type(xtik)!=type(None)):
#     if(len(xtik)==2):
#       xtik_vals,xtik_labs=xtik
#       plt.xticks(xtik_vals,xtik_labs,rotation=x_lab_rotation)
#     else:
#       plt.xticks(xtik,rotation=x_lab_rotation) 

#   #print('len(ytik)=',len(ytik))
#   if(type(ytik)!=type(None)):
#     if(len(ytik)==2):
#       ytik_vals,ytik_labs=ytik
#       plt.yticks(ytik_vals,ytik_labs,rotation=y_lab_rotation)
#     else:
#       plt.yticks(ytik,rotation=y_lab_rotation) 

#   #plt.yticks(range(12), ['J','F','M','A','M','J','J','A','S','O','N','D'], rotation='horizontal')

#   #print('locs=',locs)
#   #print('labels=',labels[:])

#   if(type(output)!=type(None)):
#     prefix,suffix=output
#     plt.savefig(prefix+'.'+suffix)
#     print('Image saved to ',prefix+'.'+suffix)
#   else:
#     #pylab.show()
#     plt.show()

#   return() #end shade_2d_simple


In [ ]:


# def shade_2d_latlon(data,**kwargs):
#   """
#   """
#   import inspect
#   import numpy as np
#   Diag = add_contours = cmap_check = title_check = units_check = extend_check = \
#     xyvals_check = clevs_check = False
#   n_contour = hatching = vector_arrays = output = None
#   xsize,ysize=6.0,4.0
#   title_fontsize=16.0

#   for key, value in kwargs.items():
#     if(key=='Diag'):
#       Diag=bool(value)
#     elif(key=='xysize'):
#       xsize,ysize=value
#     elif(key=='cmap'):
#       cmap=value
#       cmap_check=True
#     elif(key=='title'):
#       title=value
#       title_check=True
#     elif(key=='units'):
#       units=value
#       units_check=True
#     elif(key=='add_contours'):
#       add_contours=bool(value)
#     elif(key=='extend'):
#       extend=value
#     elif(key=='clevs'):
#       clevs=value
#       clevs_check=True
#     elif(key=='xyvals'):
#       xvals,yvals=value
#       xyvals_check=True
#     elif(key=='title_fontsize'):
#       title_fontsize=value
#     elif(key=='n_contour'):
#       n_contour=value
#     elif(key=='hatching'):
#       hatching=value
#       hatch_data,hatch_styles,hatch_levs=hatching
#     elif(key=='vector_arrays'):
#       vector_arrays=value
#     elif(key=='output'):
#       output=value
#       if(Diag): print('Generating output file.')
    
# #       (above_Mean, [None,'.'], [0,50,100])

#       if(Diag): print('shade_2d_latlon: hatching on')
#     else:
#       raise SystemExit('Dont know that key.'+__file__+' line number: '+str(inspect.stack()[0][2]))

#   #print('len(clevs)=',len(clevs))
#   #if(len(clevs)==0): del(clevs)

#   if(not cmap_check): cmap='jet'
#   if(not title_check): title='data'
#   if(not units_check): units='units'
#   if(not extend_check): extend='both'

#   import cartopy.crs as ccrs
#   from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
#   from cartopy.util import add_cyclic_point
#   import matplotlib as mpl
#   mpl.rcParams['mathtext.default'] = 'regular'
#   import matplotlib.pyplot as plt

#   fig = plt.gcf()
#   fig.set_size_inches(xsize, ysize) #default 6.0,4.0

#   ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
#   #clevs = [0,50,100,150,200,250,300,10000]
#   #clevs = [0,50,100,150,200,250,300]

#   #print('extend=',extend)

#   if(not clevs_check or type(clevs)==type(None)):
#     fill = ax.contourf(xvals, yvals, data, cmap=cmap, transform=ccrs.PlateCarree(), extend=extend)
#   else:
#     fill = ax.contourf(xvals, yvals, data, clevs, cmap=cmap, transform=ccrs.PlateCarree(), extend=extend)
#   if(add_contours): plt.contour(data,colors='black')

#   if(type(n_contour)!=type(None)):
#     nc = ax.contour(xvals, yvals, data, transform=ccrs.PlateCarree(), levels=n_contour, colors='black')
#     plt.clabel(nc, inline=False, fontsize=10, fmt='%3.0f') #, manual=manual_locations)
    
# #     (above_Mean, [None,'.'], [0,50,100])
# #   print('hatch_levs=',hatch_levs)
# #   print('hatch_styles=',hatch_styles)
# #   print('hatch_data=',hatch_data)
  
#   #print(type(hatching))
#   if(type(hatching)!=type(None)):
#   #hatch_data,hatch_styles,hatch_levs=hatching
#     #print('hello')
#     stip = ax.contourf(xvals, yvals, hatch_data, levels=hatch_levs, hatches=hatch_styles, transform=ccrs.PlateCarree(), colors='none')

#   vectors = []
#   if(type(vector_arrays)!=type(None)):
#     for cnt in range(len(vector_arrays)):
#       anotherX, anotherY, anotherdataU, anotherdataV, anotherpivot, anotherscale, anotherwidth = vector_arrays[cnt]
#       #print('anotherdataU=',anotherdataU)
#       #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
#       vectors = ax.quiver(anotherX, anotherY, anotherdataU, anotherdataV, \
#                           transform=ccrs.PlateCarree(), \
#                           pivot=anotherpivot, \
#                           scale=anotherscale, \
#                           width=anotherwidth) #, width=0.022)
# #                           transform=ccrs.PlateCarree(), pivot='mid', scale=500) #, width=0.022)
      
#   ax.coastlines()
#   ax.gridlines()
#   ax.set_xticks([0, 60, 120, 180, 240, 300, 359.99], crs=ccrs.PlateCarree())
#   ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
#   lon_formatter = LongitudeFormatter(zero_direction_label=True, number_format='.0f')
#   lat_formatter = LatitudeFormatter()
#   ax.xaxis.set_major_formatter(lon_formatter)
#   ax.yaxis.set_major_formatter(lat_formatter)

#   plt.colorbar(fill, orientation='horizontal')

#   if(type(units)==type(None)):
#     plt.title(title, fontsize=title_fontsize)
#   else:
#     plt.title(title+' ('+units+')', fontsize=title_fontsize)
#   #plt.savefig('test.png')

#   if(type(output)!=type(None)):
#     prefix,suffix=output
#     plt.savefig(prefix+'.'+suffix)
#     print('Image saved to ',prefix+'.'+suffix)
# #   else:
#   plt.show()
  
# #   if('output' in locals()):
# #     plt.savefig(output+'.png')
# #     print('Image saved to ',output+'.png')
# #   else:
# #     plt.show()

#   return() #end shade_2d_latlon



# print('END')